# Solving the 1d Heat equation with a PINN

And now...

It's time...

For the moment you've been waiting for

1...

2...

3...

Ready?

(Please *please* make the training loop output this when training starts)

In [20]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [21]:
# This is the head conductivity or whatever
R = 0.5

## Data

In [73]:
class TemperatureDataset(Dataset):
    def __init__(self, initial_points, initial_temps, boundary_points, random_points):
        # Random points at t = 0
        self.initial_points = initial_points
        # Temperature at each initial point
        self.initial_temps = initial_temps
        # Random points at x boundary
        self.boundary_points = boundary_points
        # Random points in domain
        self.random_points = random_points

    def __len__(self):
        return len(self.random_points)
    
    def __getitem__(self, i):
        initial_point = self.initial_points[i]
        initial_temp = self.initial_temps[i].unsqueeze(dim=1)
        boundary_point = self.boundary_points[i]
        random_point = self.random_points[i]
        return initial_point, initial_temp, boundary_point, random_point

In [23]:
T_RANGE = 10
X_RANGE = 1
n_datapoints = 10

# Random points at t = 0
initial_points = torch.zeros((n_datapoints, 2))
initial_points[:, 1] = X_RANGE * torch.rand(n_datapoints)
# Temperature 1 if 1/4 <= x <= 3/4, else 0
initial_temps = torch.where(((initial_points[:, 1] >= 1/4) & (initial_points[:, 1] <= 3/4)), 1.0, 0.0)

# Random points at the x boundary
boundary_points = torch.zeros((n_datapoints, 2))
# Pick points at random times
boundary_points[:, 0] = T_RANGE * torch.rand(n_datapoints)
# Pick left or right boundary points randomly
boundary_points[:, 1] = X_RANGE * (torch.rand(n_datapoints)[:] >= 1/2).float()

# random points in (0, T_RANGE) x (0, X_RANGE)
random_points = torch.rand((n_datapoints, 2))
random_points[:, 0] *= T_RANGE
random_points[:, 1] *= X_RANGE

In [24]:
print(initial_points, initial_temps)
print(boundary_points)
print(random_points)

tensor([[0.0000, 0.3215],
        [0.0000, 0.7318],
        [0.0000, 0.2801],
        [0.0000, 0.2127],
        [0.0000, 0.8053],
        [0.0000, 0.3585],
        [0.0000, 0.1360],
        [0.0000, 0.3103],
        [0.0000, 0.4042],
        [0.0000, 0.2050]]) tensor([1., 1., 1., 0., 0., 1., 0., 1., 1., 0.])
tensor([[0.9059, 1.0000],
        [8.6501, 1.0000],
        [2.8730, 0.0000],
        [0.7269, 0.0000],
        [5.4860, 0.0000],
        [9.1013, 0.0000],
        [3.3207, 1.0000],
        [4.5460, 1.0000],
        [5.5516, 1.0000],
        [3.1209, 0.0000]])
tensor([[9.8437, 0.6500],
        [0.9324, 0.9370],
        [0.2356, 0.5513],
        [9.2116, 0.6273],
        [3.8014, 0.9777],
        [1.0089, 0.7477],
        [1.7198, 0.6221],
        [9.6129, 0.9223],
        [9.5006, 0.7240],
        [7.3226, 0.9822]])


Everything looks OK. Let's create the dataset and dataloader now.

In [25]:
batch_size = 3

In [26]:
train_dataset = TemperatureDataset(initial_points, initial_temps, boundary_points, random_points)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [27]:
for ip, it, bp, rp in train_dataloader:
    print(it)

tensor([1., 1., 1.])
tensor([1., 1., 1.])
tensor([0., 0., 0.])
tensor([0.])


Yay it's working, now it's time to set things up and start training. First I need to figure out how to take partial derivatives.

## Model and differentiation

In [28]:
class TemperatureNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            # inputs = t, x
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128,128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        logits = self.network(x)
        return logits

In [29]:
model = TemperatureNetwork()

This is how I'm supposed to do it in the training. Now I think I can start setting things up for training.

In [30]:
for ip, it, bp, rp in train_dataloader:
    # Ensure rp requires gradient
    rp.requires_grad_(True)
    
    temp_pred = model(rp)
    print(rp, temp_pred)

    # Compute dT/dt for each point in the batch
    dT_dt = torch.autograd.grad(temp_pred, rp, grad_outputs=torch.ones_like(temp_pred), create_graph=True)[0][:, 0:1]

    # Compute d2T/dx2 for each point in the batch
    dT_dx = torch.autograd.grad(temp_pred, rp, grad_outputs=torch.ones_like(temp_pred), create_graph=True)[0][:, 1:2]
    d2T_dx2 = torch.autograd.grad(dT_dx, rp, grad_outputs=torch.ones_like(dT_dx), create_graph=True)[0][:, 1:2]

    print("dT/dt:", dT_dt)
    print("d2T/dx2:", d2T_dx2)

tensor([[1.7198, 0.6221],
        [7.3226, 0.9822],
        [9.5006, 0.7240]], requires_grad=True) tensor([[0.2157],
        [0.2972],
        [0.2965]], grad_fn=<AddmmBackward0>)
dT/dt: tensor([[0.0208],
        [0.0038],
        [0.0029]], grad_fn=<SliceBackward0>)
d2T/dx2: tensor([[-0.0564],
        [-0.0243],
        [-0.0252]], grad_fn=<SliceBackward0>)
tensor([[9.8437, 0.6500],
        [3.8014, 0.9777],
        [9.6129, 0.9223]], requires_grad=True) tensor([[0.2951],
        [0.2752],
        [0.3027]], grad_fn=<AddmmBackward0>)
dT/dt: tensor([[0.0028],
        [0.0110],
        [0.0025]], grad_fn=<SliceBackward0>)
d2T/dx2: tensor([[-0.0255],
        [-0.0322],
        [-0.0219]], grad_fn=<SliceBackward0>)
tensor([[0.9324, 0.9370],
        [1.0089, 0.7477],
        [0.2356, 0.5513]], requires_grad=True) tensor([[0.2416],
        [0.2191],
        [0.1823]], grad_fn=<AddmmBackward0>)
dT/dt: tensor([[-0.0115],
        [ 0.0033],
        [ 0.0171]], grad_fn=<SliceBackward0>)
d2T/dx2

Model is working as expected!

## Training

In [31]:
learning_rate = 3e-4
weight_decay = 1e-3
n_epochs = 10

# Loss coefficients
A, B, C = 1, 1, 10

TRAIN = False

In [32]:
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [102]:
l1_loss = nn.SmoothL1Loss(reduction='mean')

def loss_fn(initial_temp_pred, initial_temp_target, boundary_temp_pred, dT_dt, d2T_dx2):
    initial_condition_loss = A * l1_loss(initial_temp_pred, initial_temp_target)
    boundary_condition_loss = B * l1_loss(boundary_temp_pred, torch.zeros_like(boundary_temp_pred))
    
    pde_difference = dT_dt - R * d2T_dx2
    pde_loss = C * l1_loss(pde_difference, torch.zeros_like(pde_difference))
    
    return initial_condition_loss + boundary_condition_loss + pde_loss

In [103]:
ip, it, bp, rp = next(iter(train_dataloader))
i_pred = model(ip)
b_pred = model(bp)
r_pred = model(rp)

loss = loss_fn(i_pred, it.unsqueeze(dim=1), b_pred, b_pred, b_pred)
print(loss)

tensor(0.3602, grad_fn=<AddBackward0>)


In [35]:
def train_loop(model, optimizer, loss_fn, dataloader, n_epochs):
    for epoch in range(n_epochs):
        for initial_points, initial_temps, boundary_points, random_points in dataloader:
            pass